In [ ]:
import numpy as np
import MDAnalysis as mda
import os
from numpy.linalg import norm
from scipy.optimize import curve_fit
from MDAnalysis.analysis.rms import rmsd
from scipy import stats
import math
import pandas as pd
import scipy
from sympy import Plane, Point3D
from sympy.abc import x
from sympy import N

In [ ]:
os.chdir('/run/media/smuraru/Tank1/GO/ss_o1/')

In [ ]:
u = mda.Universe('md_2.tpr', 'md_2.xtc')

In [ ]:
sel_DNA = u.select_atoms('nucleic')
sel_DNA_not_backbone = u.select_atoms('nucleic and not nucleicbackbone and not resname GGG')
sel_DNA_backbone = u.select_atoms('nucleicbackbone and not resname GGG')
sel_CX = u.select_atoms('resname GGG')

In [ ]:
sel_rings = {}
sel_rings["CX"] = []
sel_rings["DNA"] = []

def gbond_CX(atom):
    gbonds = 0
    for bond in atom.bonds:
        if (bond[1].name == 'CX'):
            gbonds += 1
    return gbonds

In [ ]:
for atom in sel_CX:
    list_of_indexes = []
    if (gbond_CX(atom) >= 2):
        for bond in atom.bonds:
            if(bond[1].name=="CX" and bond[0].name=="CX"):
                if (bond[1] == atom):
                    u1 = bond[0]
                else:
                    u1 = bond[1]
                if ((gbond_CX(u1) >= 2)):
                    
                    for bond_2 in u1.bonds:
                        if(bond_2[1].name=="CX" and bond_2[0].name=="CX"):
                            if (bond_2[1] == u1):
                                u2 = bond_2[0]
                            else:
                                u2 = bond_2[1]
                            if ((gbond_CX(u2)>=2) and (u2 != atom)):
                        
                                for bond_3 in u2.bonds:
                                    if(bond_3[1].name=="CX" and bond_3[0].name=="CX"):
                                        if (bond_3[1] == u2):
                                            u3 = bond_3[0]
                                        else:
                                            u3 = bond_3[1]
                                        if ((gbond_CX(u3)>=2) and (u3!= u1)):
                                            list_of_indexes.append((u3.index, u2.index, u1.index, atom.index))
    for element in list_of_indexes:
        for element2 in list_of_indexes:
            if ((element[0] == element2[0]) and (element[1] != element2[1]) and (element[2]!=element2[2]) and (element[3] == element2[3])):
                prototype = u.select_atoms(str(" bynum " + str(element[0]+1) + " or bynum " + str(element[1]+1) + " or bynum " + str(element[2]+1) + " or bynum " + str(element[3]+1) + " or bynum " + str(element2[1]+1) + " or bynum " + str(element2[2]+1)))
                if prototype not in sel_rings["CX"]:
                    sel_rings["CX"].append(prototype)

In [ ]:
sel_rings["DNA"] = {}
for residue in sel_DNA.residues:
    sel_rings["DNA"][str(residue.resname)+"_"+str(residue.resid)] = []
#     sel_rings["DNA"][str(residue.resname)+"_"+str(residue.resid)].append(u.select_atoms('resname ' + str(residue.resname) + ' and resid ' + str(residue.resid) + " and (name C3' or name C4' or name O4' or name C1' or name C2')"))
    sel_rings["DNA"][str(residue.resname)+"_"+str(residue.resid)].append(u.select_atoms('resname ' + str(residue.resname) + ' and resid ' + str(residue.resid) + " and (name N1 or name C2 or name N3 or name C4 or name C5 or name C6)"))
    if (len(u.select_atoms('resname ' + str(residue.resname) + ' and resid ' + str(residue.resid) + " and (name C4 or name C5 or name N7 or name C8 or name N9)")) == 5):
        sel_rings["DNA"][str(residue.resname)+"_"+str(residue.resid)].append(u.select_atoms('resname ' + str(residue.resname) + ' and resid ' + str(residue.resid) + " and (name C4 or name C5 or name N7 or name C8 or name N9)"))

In [ ]:
# sel_rings["DNA"]

In [ ]:
selected = {}
for frame in u.trajectory:
    if (frame.frame % 100 == 0):
        print(frame.frame)
    selected[frame.frame] = {}
    selected[frame.frame]["DNA"] = {}
    for element in sel_rings["DNA"]:
        selected[frame.frame]["DNA"][element] = {}
        for ring in range(len(sel_rings["DNA"][element])):
            selected[frame.frame]["DNA"][element][ring] = []
            for ring2 in range(len(sel_rings["CX"])):
                if (scipy.spatial.distance.euclidean(sel_rings["DNA"][element][ring].center_of_mass(), sel_rings["CX"][ring2].center_of_mass()) <= 4.5):
                    plane1 = Plane(sel_rings["DNA"][element][ring].atoms.positions.tolist()[0], sel_rings["DNA"][element][ring].atoms.positions.tolist()[1], sel_rings["DNA"][element][ring].atoms.positions.tolist()[2])
                    plane2 = Plane(sel_rings["CX"][ring2].atoms.positions.tolist()[0], sel_rings["CX"][ring2].atoms.positions.tolist()[1], sel_rings["CX"][ring2].atoms.positions.tolist()[2])
                    angle = plane1.angle_between(plane2)
                    val = N(angle)
                    calc = math.degrees(val)
                    if ((0 <= calc <= 45) or (135 <= calc <= 180)):
                        selected[frame.frame]["DNA"][element][ring].append(sel_rings["CX"][0])
    

In [ ]:
sel_rings["CX"][0]

In [ ]:
inter_pi = {}
for frame in u.trajectory:
    inter_pi[frame.frame] = 0
    for element in sel_rings["DNA"]:
        for ring in range(len(sel_rings["DNA"][element])):
            for ring2 in range(len(selected[frame.frame]["DNA"][element][ring])):
                pair_distance = False
                for atom in sel_rings["DNA"][element][ring]:
                    for atom2 in selected[frame.frame]["DNA"][element][ring][ring2]:
                        ref = u.select_atoms("bynum " + str(int(atom.index)+1))
                        mob = u.select_atoms("bynum " + str(int(atom2.index) + 1))
                        distance = rmsd(ref.positions, mob.positions)
                        if (distance <= 4):
                            pair_distance = True
                if (pair_distance == True):
                    inter_pi[frame.frame][element] += 1
            

In [ ]:
sel_rings["DNA"]

In [ ]:
selected

In [ ]:
# inter pi stacking (CX - DNA)
inter_pi = {}
checked = []
for frame in u.trajectory:
    if (frame.frame % 50 == 0):
        print (frame.frame)
    inter_pi[frame.frame] = 0
    for element in sel_rings:
        checked.append(element)
        if (element != "CX"):
            for ring in range(len(sel_rings[element])):
                for second_ring in range(len(sel_frame[frame.frame])):
                    if (scipy.spatial.distance.euclidean(sel_rings[element][ring].center_of_mass(), sel_rings[second_element][second_ring].center_of_mass()) <= 4.5):
                        pair_distance = False
                        for atom in sel_rings[element][ring]:
                            for second_atom in sel_rings[second_element][second_ring]:
                                ref = u.select_atoms("bynum " + str(int(atom.index)+1) )
                                mob = u.select_atoms("bynum " + str(int(second_atom.index)+1))
                                min_distance = rmsd(ref.positions, mob.positions)
                                if (min_distance <= 4):
                                    pair_distance = True
                        if (pair_distance == True):
                            inter_pi[element] += 1                           

In [ ]:
# intra pi stacking (DNA-DNA)
checked = []
intra_pi = {}

for frame in u.trajectory:
    if (frame.frame % 1000 == 0):
        print(frame.frame)
    for element in sel_rings:
        checked.append(element)
        if (element != "CX"):
            intra_pi[element] = 0
            for ring in range(len(sel_rings[element])):
                for second_element in sel_rings:
                    min_dist = 0
                    pair_distance = False
                    if ((second_element != "CX") and (second_element != element) and (second_element not in checked)):
#                     print("element: ", element, " element2: ", second_element)
#                     print("element2: ", second_element)
                        for second_ring in range(len(sel_rings[second_element])):
#                         distance = math.sqrt((sel_rings[element][ring].center_of_mass()[0] - sel_rings[second_element][second_ring].center_of_mass()[0])**2 + (sel_rings[element][ring].center_of_mass()[1] - sel_rings[second_element][second_ring].center_of_mass()[1])**2 + (sel_rings[element][ring].center_of_mass()[2] - sel_rings[second_element][second_ring].center_of_mass()[2])**2)
                            if (scipy.spatial.distance.euclidean(sel_rings[element][ring].center_of_mass(), sel_rings[second_element][second_ring].center_of_mass()) <= 4.5):
                                p1 = np.array(sel_rings[element][ring].center_of_geometry())
                                p2 = np.array(sel_rings[element][ring].center_of_geometry())
                                p3 = np.array(sel_rings[element][ring].center_of_geometry())
                                p1_2 = np.array(sel_rings[second_element][second_ring].center_of_geometry())
                                p2_2 = np.array(sel_rings[second_element][second_ring].center_of_geometry())
                                p3_2 = np.array(sel_rings[second_element][second_ring].center_of_geometry())
                                v1 = p3 - p1
                                v2 = p2 - p1
                                cp = np.cross(v1, v2)
                                a, b, c = cp
                                d = np.dot(cp, p3)
#                             print("Cp = ", cp, " d= ", d)
                                pair_distance = False
                                for atom in sel_rings[element][ring]:
                                    for second_atom in sel_rings[second_element][second_ring]:
                                        ref = u.select_atoms("bynum " + str(int(atom.index)+1) )
                                        mob = u.select_atoms("bynum " + str(int(second_atom.index)+1))
                                        min_distance = rmsd(ref.positions, mob.positions)
                                   # if (scipy.sqrt((atom.position()[0] - second_atom.position()[0])**2 + (atom.position()[1] - second_atom.position()[1])**2 + (atom.position()[2] - second_atom.position()[2])**2) < 4):
                                        if (min_distance <= 4):
#                                         print(element, second_element, distance, min_distance)
                                            pair_distance = True
                            if (pair_distance == True):
                                intra_pi[element] += 1
                          #  print(element, ring, second_element, second_ring)
                                    
print(intra_pi)       
        

            
    

In [ ]:
selected["DNA"]["DC5_1231"][0]

In [ ]:
for frame in u.trajectory:
    for element in sel_rings:
        

In [ ]:
for frame in u.trajectory:
    print(inter_pi[frame.frame])  

In [ ]:
frame.frame

In [ ]:
inter_pi

In [ ]:
o = u.select_atoms("bynum 1")
o[0].index

In [ ]:
import csv


In [ ]:
with open('pi_stacking.csv', mode='w') as dist_file:
    writer = csv.writer(dist_file, delimiter=' ', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    for frame in u.trajectory:
        writer.writerow([frame_x[int(frame.frame)], dist_bkb[int(frame.frame)]])
        if (int(frame.frame) % 1000 == 0):
            print("frame: ", int(frame.frame), "out of ", len(u.trajectory))